In [1]:
import requests as rq
import json
import copy

In [2]:
r = rq.get("http://api.localhost/matches")
j = json.loads(r.text)
j[0]

{u'RI_A': -43.389652653015965,
 u'RI_D': -38.70231231092473,
 u'RI_H': -49.38146126629609,
 u'_id': u'2020-2-14/21:00/Wolverhampton - Leicester',
 u'date': u'2020-2-14',
 u'hour': u'21:00',
 u'league': u'premierLeague',
 u'match': u'Wolverhampton - Leicester',
 u'odd_A': 2.85,
 u'odd_D': 3.25,
 u'odd_H': 2.45,
 u'pred_A': 0.0,
 u'pred_D': 0.0,
 u'pred_H': 0.0,
 u'teamA': u'Leicester',
 u'teamH': u'Wolverhampton'}

In [12]:
matches = []
ecart_zero = 0.3
risk = 0
tot = 0
montant = 20
match_to_print = []
esp = 0
var =0
cred = 0
for match in j:
    if (match['RI_A'] > 30):
        match['side'] = "A"
        match['esp'] = match['pred_A'] * (match['odd_A'] - 1) - (1 - match['pred_A']) * (-1);
        match['var'] = match['pred_A'] * (match['odd_A'] - 1 - match['esp']) ** 2 + (1 - match['pred_A']) * (-1 - match['esp']) ** 2 + ecart_zero;
        match['cred'] = match['esp']/match['var']**(1 + (5-risk)/5);
        tot += match['cred']
        matches.append(copy.deepcopy(match))
    if (match['RI_D'] > 30):
        match['side'] = "D"
        match['esp'] = match['pred_D'] * (match['odd_D'] - 1) - (1 - match['pred_D']) * (-1);
        match['var'] = match['pred_D'] * (match['odd_D'] - 1 - match['esp']) ** 2 + (1 - match['pred_D']) * (-1 - match['esp']) ** 2 + ecart_zero;
        match['cred'] = match['esp']/match['var']**(1 + (5-risk)/5);
        tot += match['cred']
        matches.append(copy.deepcopy(match))
    if (match['RI_H'] > 30):
        match['side'] = "H"
        match['esp'] = match['pred_H'] * (match['odd_H'] - 1) - (1 - match['pred_H']) * (-1);
        match['var'] = match['pred_H'] * (match['odd_H'] - 1 - match['esp']) ** 2 + (1 - match['pred_H']) * (-1 - match['esp']) ** 2 + ecart_zero;
        match['cred'] = match['esp']/match['var']**(1 + (5-risk)/5);
        tot += match['cred']
        matches.append(copy.deepcopy(match))

        
for match in matches:
    match['cred'] = match['cred'] / tot
    esp += match['esp'] * match['cred']
    var +=  match['var'] * match['cred']**2
    print(match['match'] + " : " + match['side'] + " bet =", match['cred'] * montant, " gain= ", match['cred'] * match['odd_' + match['side']] * montant)
print('esperance de gain =',esp, "+/-", var)
print('intervalle : ',esp-var, " - " ,esp + var)


(u'Leganes - Betis : A bet =', 12.890464944215744, ' gain= ', 38.67139483264723)
(u'Milan AC - Torino : A bet =', 0.9464382619383105, ' gain= ', 4.968800875176131)
(u'Betis - Majorque : A bet =', 0.5934815573599271, ' gain= ', 3.5015411884235705)
(u'FC Barcelone - Eibar : A bet =', 0.027347649863980986, ' gain= ', 0.31449797343578134)
(u'Bayer Leverkusen - Augsbourg : A bet =', 0.4677300396995353, ' gain= ', 2.8531532421671653)
(u'Wolfsbourg - Mayence : A bet =', 1.5168490768301088, ' gain= ', 6.9775057534185)
(u'Manchester City - West Ham : A bet =', 0.003975281323652587, ' gain= ', 0.06757978250209398)
(u'Lille - Toulouse : A bet =', 0.25845474658705875, ' gain= ', 1.8091832261094112)
(u'Roma - Lecce : A bet =', 0.1468906435501574, ' gain= ', 1.1751251484012593)
(u'Borussia Dortmund - Fribourg : A bet =', 0.19958984887534742, ' gain= ', 1.4969238665651057)
(u'Mayence - SC Paderborn 07 : A bet =', 2.813038969401702, ' gain= ', 11.111503929136724)
(u'Hoffenheim - Bayern Munich : H bet 